## Exercise 1

In [1]:
import pandas as pd

In [ ]:
data = pd.read_csv('./corpus/labels/train_labels.csv')
data['label'] = data['kss'] > 7.5
data['label'] = data['label'].astype(int)
data.to_csv('./corpus/labels/train_labels_transformed.csv', index=False)
data.head()

In [ ]:
data = pd.read_csv('./corpus/labels/dev_labels.csv')
data['label'] = data['kss'] > 7.5
data['label'] = data['label'].astype(int)
data.to_csv('./corpus/labels/dev_labels_tranformed.csv', index=False)
data.head()

In [ ]:
data.describe()

## Exercise 2

In [2]:
import glob
import librosa
import numpy as np
from datetime import datetime, timedelta

In [ ]:
def summary(path, data):
#     print(data['Gender'].value_counts())
#     print('\n\nM1:\n', data[(data['Gender'] == 'M') & (data['label'] == 1)].count())
#     print('\n\nM0:\n', data[(data['Gender'] == 'M') & (data['label'] == 0)].count())
#     print('\n\nF1:\n', data[(data['Gender'] == 'F') & (data['label'] == 1)].count())
#     print('\n\nF0:\n', data[(data['Gender'] == 'F') & (data['label'] == 0)].count())
    
    durations = []
    for p in glob.glob(f'{path}/*.wav'):
        durations.append(librosa.get_duration(filename=p))

    print('Mean: ', np.mean(durations))
    
    d = datetime(1,1,1) + timedelta(seconds=int(sum(durations)))
    print("HOURS:MIN:SEC")
    print("%d:%d:%d" % (d.hour, d.minute, d.second))

In [ ]:
summary('./corpus/wav/test', None)

#### Results:
|       | Speakers | | SL  recordings |     | NSL  recordings |     | Average  duration (s) | Total  duration (h:m:s) |
|-------|----------|----------|----------------|-----|-----------------|-----|-----------------------|-------------------------|
|       | F        | M        | F              | M   | F               | M   |                       |                         |
| Train | 1991     | 706      | 825            | 247 | 1166            | 459 | 9.0717                | 6:47:46                 |
| Dev   | 967      | 372      | 294            | 194 | 673             | 178 | 7.8596                | 2:55:24                 |
| Test  |          |          |                |     |                 |     | 10.1975               | 3:31:15                 |

#### Comments:
* Train set is twice the size of the dev set
    * train: 35% males are sleepy and 41% of femaes are sleepy
    * dev: 52% males are sleepy 30% of females are sleepy

* Proportions in the train/dev set for males and females are almost the same
* There are more females in the datasets compared to males ~70% females
* There are more not sleepy recordings than sleepy ones

* Average recording length of the train set is closer to test set then to dev set which is smaller


* Train samples: 2697 (51%)
* Dev samples: 1339   (25%)
* Test samples: 1243  (24%)

In [3]:
len(list(glob.glob('./corpus/wav/train/*.wav'))), len(list(glob.glob('./corpus/wav/dev/*.wav'))), len(list(glob.glob('./corpus/wav/test/*.wav')))

(2697, 1339, 1243)

## Exercise 3

In [15]:
import os
from tqdm.notebook import tqdm

In [4]:
opensmile_path = '/Users/martin/Downloads/opensmile-2.3.0'
opensmile_executable_path = f'{opensmile_path}/inst/bin/SMILExtract'

In [13]:
config_file_path = '/Users/martin/Downloads/opensmile-2.3.0/config/gemaps/eGeMAPSv01a.conf'
path = './corpus/wav/test'
for p in tqdm(glob.glob(f'{path}/*.wav')):
    out_path = p[:-4] + '_eGeMAPS.csv'
    os.system(f'{opensmile_executable_path} -C {config_file_path} -I {p} -csvoutput {out_path}')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
config_file_path = '/Users/martin/Downloads/opensmile-2.3.0/config/IS11_speaker_state.conf'
path = './corpus/wav/test'
for p in tqdm(glob.glob(f'{path}/*.wav')):
    out_path = p[:-4] + '_IS11.csv'
    os.system(f'{opensmile_executable_path} -C {config_file_path} -I {p} -csvoutput {out_path}')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


#### Comments:
* eGeMAPS = 88 features + name;frameTime
* IS11 = 4368 features + name;frameTime
* Features are all functionals

## Exercise 4

In [26]:
csvs = []
for f in tqdm(glob.glob('./corpus/wav/test/*_IS11.csv')):
    df = pd.read_csv(f, sep=';')
    df['name'] = f.split('/')[-1].replace('.csv', '')
    csvs.append(df)

combined = pd.concat( csvs )
combined.to_csv('./corpus/is11_test.csv', index=False )